In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #visualization
import plotly.express as px
import seaborn as sns #visualization
from plotly.offline import init_notebook_mode, iplot, plot

Import Data

In [ ]:
df = pd.read_csv('new_data.csv')
df.head()

## **Preprocessing data**

The data set doesn't have any null values.
It has some duplicate entries that will be removed.

Also we can convert the songs duration to seconds for better readibilty.

In [ ]:
no_rows = df.shape[0]
df = df.drop_duplicates() # keep first duplicate row
df["duration_s"] = df.duration_ms.apply(lambda x : round(x/1000)) # add duration in seconds
df.drop("duration_ms", axis=1, inplace=True) #remove duration in ms
print("Dropped ", (no_rows-df.shape[0]), " duplicate rows" )
print("New dataset shape")
df.shape

In [ ]:

#Top Hits by Year
songs_per_years = df['year'].value_counts().sort_index()

iplot(px.bar(songs_per_years, 
       title='Songs per year',
       labels=dict(index='year',value='number of songs'),
       color_discrete_sequence=['#0055FF']
      ).update_xaxes(type='category'))


Since the years 1998, 1999 and 2020 have few examples, we will be removing songs from those years.
Also, we considered that the song **title** won't have any correlation with its popularity, so that column will be removed.


```
# This is formatted as code
```



In [ ]:
df = df[(df.year >= 2000) & (df.year <= 2019)] 
df = df.drop(['song'], axis = 1) #removing song title column
#df.head()


In [ ]:
#df.info()

## Popularity classification
First let's analise the popularity distribution, and try to indentify existing outliers 

In [ ]:
df.hist(column=('popularity'),bins=25, grid=False, figsize=(12,8), color='#86bf91', zorder=2, rwidth=0.9)


In [ ]:
df = df[(df.popularity > 5)]

In [ ]:
df.hist(column=('popularity'),bins=25, grid=False, figsize=(12,8), color='#86bf91', zorder=2, rwidth=0.9)

### Discretization 
- 11-62 not popular
- 62-72 semi-popular
- 72-89 popular

In [ ]:
pop, bins = pd.qcut(df.popularity, 3, labels=["not popular", "semi-popular", "popular"], retbins = True)

print("Bins: ", bins)
pop.hist()

Adding the new class, **popularity_class**, based on popularity values

In [ ]:
conditions = [
    (df['popularity'] <= 62),
    (df['popularity'] > 62) & (df['popularity'] <= 72),
    (df['popularity'] > 72)
    ]

labels = ["not popular", "semi-popular", "popular"]

#creating a new column
df['popularity_class'] = np.select(conditions, labels)


In [ ]:
df.head()


#### Pairplot
Para nos ajudar a ver possíveis erros nos dados, e se existe alguma correlação óbvia entre as diferentes colunas

In [ ]:
df['popularity_class'].unique()
num_var= ['year', 'danceability', 'energy', 'key','speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_s']

#sns.pairplot(df, hue = 'popularity_class', vars= num_var)

## Análise exploratória

- Como é que os dados estão distribuídos
- Existem correlações nos dados, e o porquê de estas existirem

Para verificar possíveis correlações vamos usar um heatmap

In [ ]:
# Correlation between features
corr = df.corr()
#Heatmap
sns.heatmap(corr, cmap="vlag")


Observando o mapa, verificamos algumas correlações positivas entre:
- Loudness e Energy - quanto mais alta a música mais energia tem
- Valence e Danceability - músicas mais positivas tendem a ser mais dançáveis

Correlações negativas:
- Energy and Acousticness - músicas acústicas têm menos energia
- Year and Duration - ao longo dos anos as músicas tendem a ser mais curtas

## Classificação

Since some of the columns have string data, it's necessary to encode it

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#all colums except popularity and popularity_class
le = preprocessing.LabelEncoder()
le.fit(df.artist)
encoded = df
encoded['artist'] = le.transform(df.artist)

le = preprocessing.LabelEncoder()
le.fit(df.genre)
encoded = df
encoded['genre'] = le.transform(df.genre)

cols = (encoded.columns.values.tolist())
cols.remove('popularity')
cols.remove('popularity_class')

#all inputs
inputs =df[cols].values

#all popularity_class labels
all_labels = df['popularity_class'].values

print("Encoded data:")
df.head()

The data is ready to be divided into training and testing sets.

In [ ]:
(training_inputs,
 testing_inputs,
 training_classes,
 testing_classes) = train_test_split(inputs, all_labels, test_size=0.25, random_state=1)

## Decision Tree
Para já vamos usar um classificador básico, ou seja, sem nenhuns ajustes nos parâmetros

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import time

dt_classifier = DecisionTreeClassifier()

parameter_grid = {
                'criterion': ['gini'],
                'splitter': ['best','random'],
                'random_state': [None,1],
                }

cross_validation = StratifiedKFold(n_splits=3)

grid_search = GridSearchCV(dt_classifier,
                           param_grid=parameter_grid,
                           cv=cross_validation)

# start_time = time.time()
# grid_search.fit(inputs, all_labels)
# print('Best score: {}'.format(grid_search.best_score_))
# print('Best parameters: {}'.format(grid_search.best_params_))
# print("Time to train: %s seconds" % (time.time() - start_time))

In [ ]:
# Create the classifier
dt_classifier = DecisionTreeClassifier(criterion='gini',random_state=None, splitter='random')

# Train the classifier on the training set
dt_classifier.fit(training_inputs, training_classes)

# Validate the classifier on the testing set using classification accuracy
dt_classifier.score(testing_inputs, testing_classes)

## LinearSVC

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

linearSVC_classifier = make_pipeline(StandardScaler(),
                     LinearSVC(random_state=0, tol=1e-5, max_iter=10e5))

linearSVC_classifier.fit(training_inputs, training_classes)
linearSVC_classifier.score(testing_inputs,testing_classes)

## K Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import time


knn_classifier = KNeighborsClassifier()

parameter_grid = {
                'n_neighbors': [x for x in range(5,40,5)],
                'weights': ['uniform','distance'],
                'algorithm': ['auto','ball_tree','kd_tree','brute'],
                'leaf_size': [x for x in range(20,40,5)],
                'p': [1,2],
                'n_jobs': [-1],
                }

cross_validation = StratifiedKFold(n_splits=10)

grid_search = GridSearchCV(knn_classifier,
                           param_grid=parameter_grid,
                           cv=cross_validation)

# start_time = time.time()
# grid_search.fit(inputs, all_labels)
# print('Best score: {}'.format(grid_search.best_score_))
# print('Best parameters: {}'.format(grid_search.best_params_))
# print("Time to train: %s seconds" % (time.time() - start_time))

In [ ]:
knn_classifier = KNeighborsClassifier(algorithm='ball_tree', leaf_size=20,n_jobs=-1,n_neighbors=10,p=1,weights='distance')
knn_classifier.fit(training_inputs,training_classes)
knn_classifier.score(testing_inputs,testing_classes)

## SVC

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
SVC_classifier = make_pipeline(StandardScaler(), SVC(gamma='auto'))
SVC_classifier.fit(training_inputs, training_classes)
SVC_classifier.score(training_inputs,training_classes)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest_classifier = RandomForestClassifier()
random_forest_classifier.fit(training_inputs,training_classes)
random_forest_classifier.score(testing_inputs,testing_classes)

## Voting Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

random_forest_classifier = RandomForestClassifier()
SVC_classifier = make_pipeline(StandardScaler(), SVC(gamma='auto'))
decision_tree_classifier = DecisionTreeClassifier(criterion='gini',random_state=None, splitter='random')

voting_classifier = VotingClassifier(estimators=[
         ('rf_classifier',random_forest_classifier),('svc', SVC_classifier),('dtc', decision_tree_classifier)], voting='hard')
voting_classifier.fit(training_inputs, training_classes)
voting_classifier.score(testing_inputs,testing_classes)

## Neural Networks


In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(inputs)
scaled_inputs = scaler.transform(inputs)
scaled_training_inputs = scaler.transform(training_inputs)
scaled_testing_inputs = scaler.transform(testing_inputs)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import time


nt_classifier = MLPClassifier()

parameter_grid = {
                'hidden_layer_sizes':   ([tuple([100 for y in range(0,x)]) for x in range(1,6)]),
                'activation': ['identity','logistic','relu','tanh'],
                'solver': ['lbfgs'],
                'random_state': [None],
                'alpha': [1e-5],
                'learning_rate': ['constant','invscaling','adaptive'],
                'max_iter': [10e50],
                'early_stopping': [True]
                }

cross_validation = StratifiedKFold(n_splits=10)

grid_search = GridSearchCV(nt_classifier,
                           param_grid=parameter_grid,
                           cv=cross_validation)

# start_time = time.time()
# grid_search.fit(scaled_inputs, all_labels)
# print('Best score: {}'.format(grid_search.best_score_))
# print('Best parameters: {}'.format(grid_search.best_params_))
# print("Time to train: %s seconds" % (time.time() - start_time))

In [ ]:
nt_classifier = MLPClassifier(activation='identity', alpha=1e-05, early_stopping=False, hidden_layer_sizes=(50,),learning_rate='constant',max_iter=10e100,random_state=None,solver='lbfgs')
nt_classifier.fit(scaled_training_inputs,training_classes)
nt_classifier.score(scaled_testing_inputs,testing_classes)

## Neural Networks with Tensorflow

In [167]:
import tensorflow as tf
from sklearn import preprocessing

#all colums except popularity and popularity_class
le = preprocessing.LabelEncoder()
le.fit(all_labels)
encoded_training_classes = le.transform(training_classes)
encoded_testing_classes = le.transform(testing_classes)

tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128,input_shape=(16,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.learning_rate = 0.01
model.momentum = 0.8

model.compile(optimizer='adam', #SGD
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

model.fit(scaled_training_inputs,
          encoded_training_classes,
          batch_size=80,
          epochs=100,
          validation_data=(scaled_testing_inputs, encoded_testing_classes))

test_loss, test_acc = model.evaluate(scaled_testing_inputs,
                                     encoded_testing_classes, verbose=2)

print('\nTest accuracy:', test_acc)
#Best: 0.332851 using {'batch_size': 80, 'epochs': 100}
#Best: 0.327004 using {'optimizer': 'SGD'}
#Best: 0.331731 using {'learn_rate': 0.01, 'momentum': 0.8}
#Best: 0.330343 using {'init_mode': 'he_uniform'}
#Best: 0.344265 using {'activation': 'linear'}

Epoch 1/100
17/17 [==============================] - 1s 14ms/step - loss: 1.0816 - accuracy: 0.4334 - val_loss: 1.1015 - val_accuracy: 0.4060
Epoch 2/100
17/17 [==============================] - 0s 7ms/step - loss: 1.0099 - accuracy: 0.4907 - val_loss: 1.0812 - val_accuracy: 0.4200
Epoch 3/100
17/17 [==============================] - 0s 5ms/step - loss: 0.9936 - accuracy: 0.5023 - val_loss: 1.0706 - val_accuracy: 0.4455
Epoch 4/100
17/17 [==============================] - 0s 6ms/step - loss: 0.9769 - accuracy: 0.5093 - val_loss: 1.0741 - val_accuracy: 0.4455
Epoch 5/100
17/17 [==============================] - 0s 6ms/step - loss: 0.9658 - accuracy: 0.5333 - val_loss: 1.0751 - val_accuracy: 0.4385
Epoch 6/100
17/17 [==============================] - 0s 6ms/step - loss: 0.9498 - accuracy: 0.5519 - val_loss: 1.0660 - val_accuracy: 0.4339
Epoch 7/100
17/17 [==============================] - 0s 5ms/step - loss: 0.9443 - accuracy: 0.5542 - val_loss: 1.0844 - val_accuracy: 0.4292
Epoch 8/100


## Tuning Batch Size and Number of Epochs

In [ ]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=inputs.shape[1], activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

numpy.random.seed(7)

model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

# grid_result = grid.fit(scaled_inputs, all_labels)

# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

## Tuning the Training Optimization Algorithm

In [ ]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=inputs.shape[1], activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
# grid_result = grid.fit(scaled_inputs, all_labels)
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

# Tuning Learning Rate and Momentum

In [ ]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=inputs.shape[1], activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
# grid_result = grid.fit(scaled_inputs, all_labels)
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

# Tuning Network Weight Initialization

In [ ]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(init_mode='uniform'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=inputs.shape[1], kernel_initializer=init_mode, activation='relu'))
	model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(scaled_inputs, all_labels)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

KeyboardInterrupt: 

# Tuning the Neuron Activation Function

In [ ]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(activation='relu'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=inputs.shape[1], kernel_initializer='uniform', activation=activation))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(scaled_inputs, all_labels)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Tuning Dropout Regularization

In [ ]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
# Function to create model, required for KerasClassifier
def create_model(dropout_rate=0.0, weight_constraint=0):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=inputs.shape[1], kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(weight_constraint)))
	model.add(Dropout(dropout_rate))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(scaled_inputs, all_labels)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Tuning the Number of Neurons in the Hidden Layer

In [ ]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
# Function to create model, required for KerasClassifier
def create_model(neurons=1):
	# create model
	model = Sequential()
	model.add(Dense(neurons, input_dim=inputs.shape[1], kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(4)))
	model.add(Dropout(0.2))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(scaled_inputs, all_labels)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))